##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Lite Model Analyzer

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/guide/model_analyzer"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/guide/model_analyzer.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">在 Google Colab 运行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/guide/model_analyzer.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/guide/model_analyzer.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>

TensorFlow Lite Model Analyzer API 能够通过列出模型的结构，帮助您分析 TensorFlow Lite 格式的模型。


## Model Analyzer API

以下 API 可用于 TensorFlow Lite Model Analyzer。

```
tf.lite.experimental.Analyzer.analyze(model_path=None,
                                      model_content=None,
                                      gpu_compatibility=False)
```

您可以在 https://www.tensorflow.org/api_docs/python/tf/lite/experimental/Analyzer 查看 API 详细信息，也可以在 Python 终端运行 `help(tf.lite.experimental.Analyzer.analyze)`。


## 简单 Keras 模型的基本用法

以下代码显示了 Model Analyzer 的基本用法。它在 TFLite 模型内容中显示转换后的 Keras 模型的内容，格式化为平面缓冲区对象。

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(128, 128)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model)


## MobileNetV3Large Keras 模型的基本用法

此 API 适用于 MobileNetV3Large 等大型模型。由于输出很大，您可能希望使用您最喜欢的文本编辑器来浏览它。

In [ ]:
model = tf.keras.applications.MobileNetV3Large()
fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model)

## 检查 GPU 委托兼容性

Model Analyzer API 通过提供  `gpu_compatibility=True` 选项，提供了一种检查给定模型的 [GPU 委托](https://www.tensorflow.org/lite/performance/gpu)兼容性的方法。


### 第 1 种情况：当模型不兼容时

以下代码展示了将 `gpu_compatibility=True` 选项用于简单 tf.function 的方式，该函数使用与 GPU 委托不兼容的带有二维张量的 `tf.slice` 和 `tf.cosh`。

对于每个存在兼容性问题的节点，您将看到 `GPU COMPATIBILITY WARNING`。

In [ ]:
import tensorflow as tf

@tf.function(input_signature=[
    tf.TensorSpec(shape=[4, 4], dtype=tf.float32)
])
def func(x):
  return tf.cosh(x) + tf.slice(x, [1, 1], [1, 1])

converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [func.get_concrete_function()], func)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
fb_model = converter.convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model, gpu_compatibility=True)

### 第 2 种情况：当模型兼容时

在本示例中，给定的模型与 GPU 委托兼容。

**注**：即使该工具没有发现任何兼容性问题，它也不能保证您的模型在每台设备上都能很好地使用 GPU 委托。可能会发生一些运行时不兼容的情况，例如目标 OpenGL 后端缺少 `CL_DEVICE_IMAGE_SUPPORT` 功能。


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(128, 128)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model, gpu_compatibility=True)